In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('out_train_v2.csv')

In [3]:
df1 = pd.read_csv('out_test_v2.csv')

In [4]:
X = df.drop("WnvPresent", axis=1)
y = df.WnvPresent

In [5]:
X.head()
X = X.drop(['Unnamed: 0', 'Date', 'Address', 'Block', 'Street', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy', 'Tavg', 'Tmin', 'NumMosquitos', 'DewPoint'], axis=1)

In [6]:
test = df1.drop(['Unnamed: 0','Id', 'Date', 'Address', 'Block', 'Street', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy', 'Tavg', 'Tmin', 'DewPoint'], axis=1)

In [7]:
X.head()

,Species,Trap,Tmax,PrecipTotal,AvgSpeed,length_of_day
0,CULEX PIPIENS/RESTUANS,T002,88,0.0,6.5,896.0
1,CULEX RESTUANS,T002,88,0.0,6.5,896.0
2,CULEX RESTUANS,T007,88,0.0,6.5,896.0
3,CULEX PIPIENS/RESTUANS,T015,88,0.0,6.5,896.0
4,CULEX RESTUANS,T015,88,0.0,6.5,896.0


In [8]:
test.head()

,Species,Trap,Tmax,PrecipTotal,AvgSpeed,length_of_day
0,CULEX PIPIENS/RESTUANS,T002,86,0.0,10.0,910.0
1,CULEX RESTUANS,T002,86,0.0,10.0,910.0
2,CULEX PIPIENS,T002,86,0.0,10.0,910.0
3,CULEX SALINARIUS,T002,86,0.0,10.0,910.0
4,CULEX TERRITANS,T002,86,0.0,10.0,910.0


In [9]:
X.Species = X.Species.astype('category')
X.Species = X.Species.cat.codes

In [10]:
test.Species = test.Species.astype('category')
test.Species = test.Species.cat.codes

In [11]:
X.dtypes

Species             int8
Trap              object
Tmax               int64
PrecipTotal      float64
AvgSpeed         float64
length_of_day    float64
dtype: object

In [12]:
X.Trap = X.Trap.astype('category')
X.Trap = X.Trap.cat.codes

In [13]:
test.Trap = test.Trap.astype('category')
test.Trap = test.Trap.cat.codes

In [14]:
X.dtypes

Species             int8
Trap               int16
Tmax               int64
PrecipTotal      float64
AvgSpeed         float64
length_of_day    float64
dtype: object

In [15]:
test.dtypes

Species             int8
Trap               int16
Tmax               int64
PrecipTotal      float64
AvgSpeed         float64
length_of_day    float64
dtype: object

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [17]:
X = StandardScaler().fit_transform(X)

In [18]:
test = StandardScaler().fit_transform(test)

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report, roc_curve, auc

In [20]:
knn = KNeighborsClassifier(n_neighbors = 2)

In [21]:
names = ["present", "not present", "predicted present", "predicted not present"]

In [23]:
def evaluate_model(model, X, y, names, test):
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=7)
    model = model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    #print model.score(X_test, Y_test)
    Y_pp = pd.DataFrame(model.predict_proba(X_test), columns=['class_0_pp','class_1_pp'])
    Y_pp['pred_class_thresh10'] = [1 if x >= 0.05 else 0 for x in Y_pp.class_1_pp.values]
    X_test_df = pd.DataFrame(X_test)
    conmat = np.array(confusion_matrix(Y_test, Y_pp.pred_class_thresh10, labels=[1,0]))
    confusion = pd.DataFrame(conmat, index=[names[0:2]],
                         columns=[names[2:]]) 
    print(Y_pp.iloc[0:10])
    print confusion
    print(classification_report(Y_test, Y_pp.pred_class_thresh10))
    print Y_pp.shape
    print X_test_df.head()
    
    Y_score = model.decision_function(X_test)

    FPR = dict()
    TPR = dict()
    ROC_AUC = dict()

# For class 1, find the area under the curve
    FPR[1], TPR[1], _ = roc_curve(Y_test, Y_score)
    ROC_AUC[1] = auc(FPR[1], TPR[1])

# Plot of a ROC curve for class 1 (has_cancer)
    plt.figure(figsize=[11,9])
    plt.plot(FPR[1], TPR[1], label='ROC curve (area = %0.2f)' % ROC_AUC[1], linewidth=4)
    plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.title('Receiver operating characteristic for cancer detection', fontsize=18)
    plt.legend(loc="lower right")
    plt.show()
    return "cross val mean score is", cross_val_score(model, X_train, Y_train, cv=5).mean()
    
    


evaluate_model(knn, X, y, names, test)

   class_0_pp  class_1_pp  pred_class_thresh10
0         0.0         1.0                    1
1         1.0         0.0                    0
2         1.0         0.0                    0
3         1.0         0.0                    0
4         1.0         0.0                    0
5         1.0         0.0                    0
6         1.0         0.0                    0
7         1.0         0.0                    0
8         1.0         0.0                    0
9         1.0         0.0                    0
             predicted present  predicted not present
present                     53                    129
not present                298                   2987
             precision    recall  f1-score   support

          0       0.96      0.91      0.93      3285
          1       0.15      0.29      0.20       182

avg / total       0.92      0.88      0.89      3467

(3467, 3)
          0         1         2         3         4         5
0 -1.177499  1.386010  0.501572 -0

AttributeError: 'KNeighborsClassifier' object has no attribute 'decision_function'

In [24]:
model = knn.fit(X, y)
y_pred = model.predict(test)

In [ ]:
len(y_pred)

In [ ]:
evaluate_model(knn, X1, y, names)

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, max_features='sqrt')


In [ ]:
evaluate_model(rf, X, y, names, test)

In [ ]:
evaluate_model(rf, X1, y, names)

In [ ]:
#gradient boosting
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(max_features="sqrt", max_depth=3, random_state=42)

In [ ]:
svc = SVC(kernel='linear')

In [ ]:
model = gbc.fit(X, y)
y_pred = model.predict_proba(test)

In [ ]:
y_preds = pd.DataFrame(data=y_pred, columns=["WnvNotPresent", "WnvPresent"])

In [ ]:
y_preds.head()

In [ ]:
df_out = pd.merge(df1, y_preds[['WnvPresent']], how = 'left', left_index=True, right_index=True )

In [ ]:
df_out = df_out[['Id', 'WnvPresent']]

In [ ]:
df_out.head()


In [ ]:
df_out.WnvPresent.value_counts()

In [ ]:
df_out.to_csv('evaluation1.csv', ignore_index=True)

In [ ]:
evaluate_model(gbc, X, y, names)

In [ ]:
evaluate_model(gbc, X1, y, names)

## classifiers that don't work

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

#using support vector machine to classify. Have set the kernel to poly as polynomial kernels are better for natural language processing
#have set degree to 2 to prevent overfitting

lr = LogisticRegression()
svc = SVC(kernel='poly')

In [ ]:
evaluate_model(lr, X, y, names)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=7)
model = svc.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print model.score(X_test, Y_test)
# Y_pp = pd.DataFrame(model.predict_proba(X_test), columns=['class_0_pp','class_1_pp'])
# Y_pp['pred_class_thresh10'] = [1 if x >= 0.05 else 0 for x in Y_pp.class_1_pp.values]
# X_test_df = pd.DataFrame(X_test)
conmat = np.array(confusion_matrix(Y_test, Y_pred, labels=[1,0]))
confusion = pd.DataFrame(conmat, index=[names[0:2]],
                         columns=[names[2:]]) 
# print(Y_pp.iloc[0:10])
print confusion
print(classification_report(Y_test, Y_pred))
# print Y_pp.shape
# print X_test_df.head()
    
Y_score = model.decision_function(X_test)

FPR = dict()
TPR = dict()
ROC_AUC = dict()

# For class 1, find the area under the curve
FPR[1], TPR[1], _ = roc_curve(Y_test, Y_score)
ROC_AUC[1] = auc(FPR[1], TPR[1])

# Plot of a ROC curve for class 1 (has_cancer)
plt.figure(figsize=[11,9])
plt.plot(FPR[1], TPR[1], label='ROC curve (area = %0.2f)' % ROC_AUC[1], linewidth=4)
plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.title('Receiver operating characteristic for cancer detection', fontsize=18)
plt.legend(loc="lower right")
plt.show()
print "cross val mean score is", cross_val_score(model, X_train, Y_train, cv=5).mean()
    

### Grid search section

In [ ]:
from sklearn.grid_search import GridSearchCV
#cvs = [1,2,3,5,10,15]
max_depth=[3,4,5,6,7]
random_state=range(10,30)
max_features=['auto','sqrt','log2',None]
#[3,15,20,50,66]
grid = GridSearchCV(estimator=gbc, param_grid=dict( max_depth=max_depth,random_state=random_state,max_features=max_features   ))
grid_coef = grid.fit(X, y)
print(grid)
grid.best_score_



In [ ]:
grid.best_params_